## Подготовка данных

In [1]:
import numpy as np
import pandas as pd
import gensim

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Загрузка предобученных векторов слов
pretrained_vectors_path = '/content/drive/MyDrive/Colab Notebooks/data/model.model.vectors.npy'

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
data = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/data/отзывы за лето.xls")
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [4]:
# Предварительная обработка текста
from sklearn.model_selection import train_test_split

# Разделение данных на обучающую и тестовую выборки
train_data, test_data, train_labels, test_labels = train_test_split(
    data['Content'], data['Rating'], test_size=0.3, random_state=42)

In [5]:
import nltk
import re

# Загрузка ресурсов NLTK (например, стоп-слов)
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Функция для предварительной обработки текста
def preprocess_text(text):
    if isinstance(text, str):  # Проверяем, что текст является строкой
        # Преобразование текста в нижний регистр
        text = text.lower()

        # Удаление символов пунктуации и специальных символов
        text = re.sub(r'[^a-zA-Zа-яА-Я0-9\s]', '', text)

        # Токенизация текста (разделение на слова)
        tokens = word_tokenize(text)

        # Удаление стоп-слов (часто встречающихся слов, которые не несут смысла)
        stop_words = set(stopwords.words('russian'))  # Список стоп-слов для русского языка
        tokens = [word for word in tokens if word not in stop_words]

        # Объединение токенов обратно в текст
        preprocessed_text = ' '.join(tokens)
    else:
        preprocessed_text = ''  # Если значение не является строкой, возвращаем пустую строку

    return preprocessed_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# Применение функции предварительной обработки к данным
train_data_preprocessed = train_data.apply(preprocess_text)
test_data_preprocessed = test_data.apply(preprocess_text)

# Прочие статические переменные
tokenizer = Tokenizer()
vocab_size = len(tokenizer.word_index) + 1  # +1 для учета паддинга (PAD)

In [7]:
#/content/drive/MyDrive/Colab Notebooks/data/
train_data_preprocessed.head()

10890                                                супер
19623    отличное приложение очень удобно управлять сче...
7430                                                    ок
7934                                             0 минусов
9630                      очень приятное положение спасибо
Name: Content, dtype: object

## Создание моделей CNN

### 2.1 Модель с предобученными векторами

В этой модели используем предобученные векторные представления слов.


In [8]:
epochs = 5  # Количество эпох (сколько раз пройти через обучающие данные)
embedding_dim = 300 # Размерность векторов
batch_size = 10  # Размер пакета (сколько примеров обрабатывать за одну итерацию)

### model 214, веса не переобучаются

# Создание токенизатора и преобразование текста в последовательности чисел
tokenizer.fit_on_texts(train_data_preprocessed)
train_sequences = tokenizer.texts_to_sequences(train_data_preprocessed)
test_sequences = tokenizer.texts_to_sequences(test_data_preprocessed)

max_length = max(len(seq) for seq in train_sequences)

# Преобразование последовательностей в одинаковой длине
train_data_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_data_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Загрузка предобученных векторов
pretrained_vectors = np.load(pretrained_vectors_path)

# Определение размера словаря (vocab_size) и размерности векторов (embedding_dim)
vocab_size, embedding_dim = pretrained_vectors.shape

# Создание модели CNN с предобученными векторами
model_pretrained_embedding = Sequential()
model_pretrained_embedding.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length, weights=[pretrained_vectors], trainable=False))
model_pretrained_embedding.add(Conv1D(128, 5, activation='relu'))
model_pretrained_embedding.add(GlobalMaxPooling1D())
model_pretrained_embedding.add(Dense(1, activation='sigmoid'))

# Компиляция модели
model_pretrained_embedding.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение модели
history = model_pretrained_embedding.fit(train_data_padded, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(test_data_padded, test_labels))

# Оценка модели
test_loss, test_accuracy = model_pretrained_embedding.evaluate(test_data_padded, test_labels)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Epoch 1/5
1447/1447 [==============================] - 42s 28ms/step - loss: -33050.2109 - accuracy: 0.1092 - val_loss: -96397.7656 - val_accuracy: 0.1125
Epoch 2/5
1447/1447 [==============================] - 42s 29ms/step - loss: -208369.2188 - accuracy: 0.1092 - val_loss: -340637.1562 - val_accuracy: 0.1125
Epoch 3/5
1447/1447 [==============================] - 42s 29ms/step - loss: -510137.9375 - accuracy: 0.1092 - val_loss: -694878.8125 - val_accuracy: 0.1125
Epoch 4/5
1447/1447 [==============================] - 41s 28ms/step - loss: -913635.7500 - accuracy: 0.1092 - val_loss: -1144461.5000 - val_accuracy: 0.1125
Epoch 5/5
194/194 [==============================] - 5s 25ms/step - loss: -1684019.8750 - accuracy: 0.1125
Test Loss: -1684019.8750
Test Accuracy: 0.1125


In [12]:
# Другая модель

# Загрузка предобученных векторов из файла
model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Colab Notebooks/data/model.bin', binary=True)

# Получение списка всех слов в словаре
all_words = model.index_to_key

# Создание словаря, который сопоставляет словам и их векторные представления
word_vectors = {word: model.get_vector(word) for word in all_words}

# Пример: Получение вектора для слова "apple" (замените на нужное вам слово)
pretrained_vector = word_vectors.get('apple', None)

if pretrained_vector is not None:
    # Определение размера словаря (vocab_size) и размерности векторов (embedding_dim)
    vocab_size, embedding_dim = len(all_words), len(pretrained_vector)

    # Создание матрицы предобученных векторов
    pretrained_vectors = np.zeros((vocab_size, embedding_dim))
    for i, word in enumerate(all_words):
        pretrained_vectors[i] = word_vectors[word]

    # Дальнейший код остается таким же, как в вашем исходном коде
else:
    print("Слово 'apple' не найдено в словаре.")

Слово 'apple' не найдено в словаре.


In [13]:
### model 182

epochs = 10  # Количество эпох (сколько раз пройти через обучающие данные)
embedding_dim = 100 # Размерность векторов
batch_size = 5  # Размер пакета (сколько примеров обрабатывать за одну итерацию)

# Создание токенизатора и преобразование текста в последовательности чисел
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data_preprocessed)
train_sequences = tokenizer.texts_to_sequences(train_data_preprocessed)
test_sequences = tokenizer.texts_to_sequences(test_data_preprocessed)

max_length = max(len(seq) for seq in train_sequences)

# Создание словаря, который сопоставляет словам их векторные представления
word_vectors = {word: model[word] for word in all_words}

# Преобразование последовательностей в одинаковой длине
train_data_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_data_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Создание матрицы предобученных векторов для вашего словаря
vocab_size = len(all_words)
embedding_dim = len(word_vectors[all_words[0]])
pretrained_vectors = np.zeros((vocab_size, embedding_dim))
for i, word in enumerate(all_words):
    pretrained_vectors[i] = word_vectors[word]

# Создание модели CNN с предобученными векторами
model_pretrained_embedding = Sequential()
model_pretrained_embedding.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length, weights=[pretrained_vectors], trainable=False))
model_pretrained_embedding.add(Conv1D(128, 5, activation='relu'))
model_pretrained_embedding.add(GlobalMaxPooling1D())
model_pretrained_embedding.add(Dense(1, activation='sigmoid'))

# Компиляция модели
model_pretrained_embedding.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение модели
history = model_pretrained_embedding.fit(train_data_padded, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(test_data_padded, test_labels))

# Оценка модели
test_loss, test_accuracy = model_pretrained_embedding.evaluate(test_data_padded, test_labels)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


Epoch 1/10
2893/2893 [==============================] - 53s 18ms/step - loss: -369238.1562 - accuracy: 0.1092 - val_loss: -1040869.2500 - val_accuracy: 0.1125
Epoch 2/10
2893/2893 [==============================] - 49s 17ms/step - loss: -2181594.2500 - accuracy: 0.1092 - val_loss: -3505891.5000 - val_accuracy: 0.1125
Epoch 3/10
2893/2893 [==============================] - 49s 17ms/step - loss: -5205779.5000 - accuracy: 0.1092 - val_loss: -7069487.0000 - val_accuracy: 0.1125
Epoch 4/10
2893/2893 [==============================] - 47s 16ms/step - loss: -9318260.0000 - accuracy: 0.1092 - val_loss: -11707197.0000 - val_accuracy: 0.1125
Epoch 5/10
2893/2893 [==============================] - 47s 16ms/step - loss: -14481677.0000 - accuracy: 0.1092 - val_loss: -17380522.0000 - val_accuracy: 0.1125
Epoch 6/10
2893/2893 [==============================] - 49s 17ms/step - loss: -20675914.0000 - accuracy: 0.1092 - val_loss: -24077748.0000 - val_accuracy: 0.1125
Epoch 7/10
2893/2893 [==============

### 2.2 Модель с обучаемыми весами Embedding

В этой модели инициализируем слой Embedding по умолчанию: trainable=True установлено для слоя Embedding, что позволяет обучать веса этого слоя вместе со всей сетью.







In [14]:
### model 214, веса переобучаются

epochs = 7  # Количество эпох (сколько раз пройти через обучающие данные)
embedding_dim = 200 # Размерность векторов
batch_size = 40  # Размер пакета (сколько примеров обрабатывать за одну итерацию)

# Загрузка предобученных векторов слов (замените на путь к вашему файлу)
pretrained_vectors_path = '/content/drive/MyDrive/Colab Notebooks/data/model.model.vectors.npy'

# Создание токенизатора и преобразование текста в последовательности чисел
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data_preprocessed)
train_sequences = tokenizer.texts_to_sequences(train_data_preprocessed)
test_sequences = tokenizer.texts_to_sequences(test_data_preprocessed)

max_length = max(len(seq) for seq in train_sequences)

# Преобразование последовательностей в одинаковой длине
train_data_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_data_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Загрузка предобученных векторов
pretrained_vectors = np.load(pretrained_vectors_path)

# Определение размера словаря (vocab_size) и размерности векторов (embedding_dim)
vocab_size, embedding_dim = pretrained_vectors.shape

# Создание модели CNN с обучаемыми весами Embedding
model_trainable_embedding = Sequential()
model_trainable_embedding.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length, weights=[pretrained_vectors], trainable=True))  # Обратите внимание на trainable=True
model_trainable_embedding.add(Conv1D(128, 5, activation='relu'))
model_trainable_embedding.add(GlobalMaxPooling1D())
model_trainable_embedding.add(Dense(1, activation='sigmoid'))

# Компиляция модели
model_trainable_embedding.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение модели
history = model_trainable_embedding.fit(train_data_padded, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(test_data_padded, test_labels))

# Оценка модели
test_loss, test_accuracy = model_trainable_embedding.evaluate(test_data_padded, test_labels)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


Epoch 1/7
362/362 [==============================] - 875s 2s/step - loss: -21353.3633 - accuracy: 0.1090 - val_loss: -88906.1484 - val_accuracy: 0.1125
Epoch 2/7
362/362 [==============================] - 871s 2s/step - loss: -341529.7812 - accuracy: 0.1092 - val_loss: -729441.8750 - val_accuracy: 0.1125
Epoch 3/7
362/362 [==============================] - 881s 2s/step - loss: -1447815.7500 - accuracy: 0.1092 - val_loss: -2360114.7500 - val_accuracy: 0.1125
Epoch 4/7
362/362 [==============================] - 879s 2s/step - loss: -3701293.5000 - accuracy: 0.1092 - val_loss: -5273913.0000 - val_accuracy: 0.1125
Epoch 5/7
362/362 [==============================] - 885s 2s/step - loss: -7348361.0000 - accuracy: 0.1092 - val_loss: -9689034.0000 - val_accuracy: 0.1125
Epoch 6/7
362/362 [==============================] - 875s 2s/step - loss: -12596903.0000 - accuracy: 0.1092 - val_loss: -15773266.0000 - val_accuracy: 0.1125
Epoch 7/7
194/194 [==============================] - 6s 29ms/step - 

## Выводы

Были рассмотрены 2 модели с сайта [rusvectores.org](https://rusvectores.org/ru/about/): [#214](http://vectors.nlpl.eu/repository/20/214.zip) и [#182](http://vectors.nlpl.eu/repository/20/182.zip).

Модель #182 работала быстрее, но численные результаты идентичные для всех рассмотренных случаев.


